# 游戏手柄避障数据收集

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event,bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from traitlets.config.configurable import Configurable
from IPython.display import display
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

## 2.连接dachbot

In [ ]:
rbt = Robot()
rbt.connect()

## 3.创建图片收集文件夹

In [ ]:
import os
free_dir = 'datasets/free'
blocked_dir = 'datasets/block'
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('该文件夹已创建！')

## 4.创建图片收集按钮

In [ ]:
# 按钮外观
button_layout = widgets.Layout(width='128px', height='64px')
# 无障碍按钮
free_button = widgets.Button(description='无障碍', button_style='success', layout=button_layout)
# 有障碍按钮
blocked_button = widgets.Button(description='有障碍', button_style='danger', layout=button_layout)
# 无障碍图片数量
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
# 有障碍图片数量
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))
# 显示图片收集按钮与每个类别的图片数量
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## 5.给按钮添加点击功能

In [ ]:
from uuid import uuid1
# 创建保存图片功能函数
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
# 保存无障碍图片
def save_free():
    global save_dir
    save_dir = free_dir
# 保存有障碍图片 
def save_blocked():
    global save_dir
    save_dir = blocked_dir
# 添加点击功能
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())
# 显示图片收集按钮与每个类别的图片数量
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## 6.在图像上创建绿色标记
1. 创建图像显示窗口；
2. 获取图像数据；
3. 在图像上创建一个绿点在标记时作为参考；
4. 显示图形化界面。

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)

def on_new_image(evt):
    image_widget.value= bgr8_to_jpeg(evt.dict['data'])
    
rbt.camera.start()
rbt.base.set_ptz(-10)
rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
display(image_widget)

### 7.创建gamepad控制器

我们可以通过游戏手柄来控制dachbot，首先将游戏手柄的无线usb插到电脑上，然后打开游戏手柄的开关。http://html5gamepad.com 打开这个网址然后按下你正在使用的游戏手柄上的按钮，可以看到网页上也会有相应的反应。
在网站上可以看到“index”下面有个数字，记住这个数字，然后我们通过“widgets.Controller()”连接到手柄然后再显示出来。

In [ ]:
controller = widgets.Controller(index=0)
display(controller)

## 8.创建手柄控制功能
1. 控制机器人前后左右运动
2. 控制机器人摄像头运动

In [ ]:
# 控制小车运动
class Move(Configurable):
    x_speed = traitlets.Float(default_value=0.0)
    a_speed = traitlets.Float(default_value=0.0)
    y_speed = traitlets.Float(default_value=0.0)
    @traitlets.observe('x_speed')
    def x_speed_value(self, change):
        self.x_speed=change['new']
        rbt.base.move(x_speed=self.x_speed*0.3, a_speed=self.a_speed, y_speed=-self.y_speed*0.5)

    @traitlets.observe('a_speed')
    def a_speed_value(self, change):
        self.a_speed=change['new']
        rbt.base.move(x_speed=self.x_speed*0.3, a_speed=self.a_speed, y_speed=-self.y_speed*0.5)
        
    @traitlets.observe('y_speed')
    def y_speed_value(self, change):
        self.y_speed=change['new']
        rbt.base.move(x_speed=self.x_speed*0.3, a_speed=self.a_speed, y_speed=-self.y_speed*0.5)
        
# 控制摄像头转动  
class Camera(Configurable):
    cx_speed = traitlets.Float(default_value=0.0)
    cy_speed = traitlets.Float(default_value=0.0)
    @traitlets.observe('cx_speed')
    def x_speed_value(self, change):
        self.cx_speed=change['new']
        rbt.base.set_ptz(x=self.cx_speed*90, y=self.cy_speed*90)

    @traitlets.observe('cy_speed')
    def a_speed_value(self, change):
        self.cy_speed=change['new']
        rbt.base.set_ptz(x=self.cx_speed*90, y=self.cy_speed*90)
        
# 实例化对象
move=Move()
camera = Camera()

# 将手柄上的摇杆axes[1]来控制小车的前进后退，摇杆axes[0]来控制小车的左右。摇杆axes[2]来控制左右平移
move_link = traitlets.dlink((controller.axes[1], 'value'), (move, 'x_speed'), transform=lambda x: -x)
turn_link = traitlets.dlink((controller.axes[0], 'value'), (move, 'a_speed'), transform=lambda x: -x)
if rbt.name=='dachbot':
    shift_left_link = traitlets.dlink((controller.buttons[6], 'value'), (move, 'y_speed'), transform=lambda x: x)
    shift_right_link = traitlets.dlink((controller.buttons[7], 'value'), (move, 'y_speed'), transform=lambda x: -x)
elif rbt.name=='dbot':
    camera_x_link = traitlets.dlink((controller.axes[2], 'value'), (camera, 'cx_speed'), transform=lambda x: x)
camera_y_link = traitlets.dlink((controller.axes[3], 'value'), (camera, 'cy_speed'), transform=lambda x: -x)

# 定义断开控制函数
def unlink_control():
    move_link.unlink()
    turn_link.unlink()
    if rbt.name=='dachbot':
        shift_left_link.unlink()
        shift_right_link.unlink()
    elif rbt.name=='dbot':
        camera_x_link.unlink()
    camera_y_link.unlink()

## 9.开始收集数据

In [ ]:
# 根据图像小部件的红色圆圈的中心坐标作为图片名称
# 通过该坐标确定当前状态应该左转和右转
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

# 创建保存无障碍按钮并连接到游戏手柄，通过点击手柄上的buttons[1]就可以保存图片了
def save_free(change):
    if change['new']:
        image_path = os.path.join(free_dir, str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        free_count.value = len(glob.glob(os.path.join(free_dir, '*.jpg')))
controller.buttons[1].observe(save_free, names='value')

# 创建保存有障碍按钮并连接到游戏手柄，通过点击手柄上的buttons[2]就可以保存图片了
def save_blocked(change):
    if change['new']:
        image_path = os.path.join(blocked_dir, str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        blocked_count.value = len(glob.glob(os.path.join(blocked_dir, '*.jpg')))        
controller.buttons[2].observe(save_blocked, names='value')

# 显示可视化窗口
display(image_widget)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## 10.断开连接
当你收集了最够多的数据后，运行下面代码与dachbot断开连接。

In [ ]:
# unlink_control()
# rbt.disconnect()